In [1]:
import nltk
import numpy as np
import pandas as pd
import re

In [ ]:
df = pd.read_csv("train.tsv", sep="\t", nrows=60000)
df.sample(5)

In [3]:
df.shape

(60000, 4)

In [4]:
df["Phrase"] = df["Phrase"].str.lower()
# df.sample(5)

In [5]:
def remove_pat(s1, pat, nstr):
    return re.sub(pat, nstr, s1)
df["Phrase"] = np.vectorize(remove_pat)(df["Phrase"], "[^a-z]+", " ")
# df.sample(5)

In [6]:
df["NewPhrase"] = np.vectorize(remove_pat)(df["Phrase"], r"\b[a-z]{1,2}\b", " ")
# df.sample(10)

In [7]:
df.Sentiment.replace([0], [1], inplace=True)
df.Sentiment.replace([1], [1], inplace=True)
df.Sentiment.replace([2], [1], inplace=True)
df.Sentiment.replace([3], [0], inplace=True)
df.Sentiment.replace([4], [0], inplace=True)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
# df = df[df["Sentiment"].isin([0,1])]
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=1, max_features=3, stop_words='english')
bow = tfidf_vectorizer.fit_transform(df['Phrase'])
bow.shape

(60000, 3)

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
X = bow
y = df["Sentiment"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)


model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)


# model = MultinomialNB()
# model = RandomForestClassifier(n_estimators=10, max_depth=550, max_leaf_nodes=None)
model.fit(X_train, y_train)

print(model.score(X_test, y_test))

yp = model.predict(X_test)
cm = confusion_matrix(yp, y_test)
y_test[y_test==1].count()
cm

0.8495


array([[ 942,  249],
       [ 654, 4155]], dtype=int64)

In [13]:
threshold = 0.4
yp2_prob = model.predict_proba(X_test)
yp2 = (yp2_prob [:,1] >= threshold).astype('int')
cm = confusion_matrix(yp2, y_test)
y_test[y_test==1].count()
cm

array([[ 850,  192],
       [ 746, 4212]], dtype=int64)